In [1]:
import json
import os
from parlai_diplomacy.utils.game2seq.factory import get_input_format, sequence_formatter_factory

from parlai_diplomacy.utils.game2seq.format_helpers.message_history import MessageHistoryBuilder
from fairdiplomacy.pydipcc import Game
from fairdiplomacy.typedefs import Power, GameJson
from parlai_diplomacy.utils.game2seq.format_helpers.misc import organize_game_by_phase
from parlai_diplomacy.utils.game2seq.dialogue_prediction import (
    DialoguePredictionFormatter,
    TrainingDialoguePredictionFormatter,
)
from fairdiplomacy.models.consts import POWERS

from parlai_diplomacy.utils.game2seq.typing import Metadata



In [8]:
# ENGLAND = "ENGLAND"
# RUSSIA = "RUSSIA"
# PHASE = "W1902A"
def load_game():
#     fle = "./data/cicero_redacted_games/game_436345_RUSSIA_AEFGIT.json"
    fle = "/data/eric_tang/diplomacy_experiments/results/diplomacy/adhoc/2023-04-14T042044.311663/c01_ag_cmp/cmp/Iag@agents_cicero.prototxt@Iag@agents_cicero.prototxt@pow@TURKEY_1d00781c/output.json"

    with open(fle, "r") as f:
        game_json = json.load(f)
    with open(fle, "r") as f:
        game_object = Game.from_json(f.read())
    # roll forward to the next phase
    last_phase_orders = game_json["phases"][-1]["orders"]
    for pwr, orders in last_phase_orders.items():
        game_object.set_orders(pwr, orders)
    game_object.process()

    return game_object


In [9]:
def _get_sequences(task_name: str, metadata: Metadata = None, **metadata_kwargs):
    game = load_game()
    input_format = get_input_format(task_name)
    if metadata is None:
        metadata = _get_metadata(**metadata_kwargs, task=task_name)
    seqs = TrainingDialoguePredictionFormatter(version=1).change_format(
        game, input_format, metadata
    )
    return seqs

def _get_metadata(
        include_player_ratings: bool = True,
        include_player_chattiness: bool = True,
        task: str = "",
    ):
        game_id = 2
        metadata = {
            "game_id": game_id,
            "pseudo_orders": {},
            "power_metadata": {},
            "opt": {
                "include_player_ratings": include_player_ratings,
                "include_player_chattiness": include_player_chattiness,
                "task": task,
                "extend_state_history_since_last_n_movement_phase": 0,
            },
        }
#         pso = "England: A LVP - YOR; F EDI - NTH; F LON - ENG [EO_O]\nFrance: A MAR - SPA; A PAR - BUR; F BRE - MAO [EO_O]\nItaly: A ROM - VEN; A VEN - TYR; F NAP - ION [EO_O]\nGermany: A BER - KIE; A MUN - RUH; F KIE - DEN [EO_O]\nTurkey: A CON - BUL; A SMY - CON; F ANK - BLA [EO_O]\nRussia: A MOS - UKR; A WAR H; F SEV - BLA; F STP/SC - BOT [EO_O]\nAustria: A BUD - SER; A VIE - BUD; F TRI - ALB [EO_O]"
#         for ph in (MOVEMENT_PHASE, MOVEMENT_PHASE_2, PHASE, MOVEMENT_PHASE_3):
#             for pow in range(1, 8):
#                 for i in range(1, 10):
#                     k = f"{game_id}-{ph}-{pow}-{i}"
#                     metadata["pseudo_orders"][k] = pso
        for power in POWERS:
            metadata["power_metadata"][power] = {"rating": 1, "chattiness": 1}

        return metadata

In [10]:
seqs = _get_sequences("message_history_orderhistorysincelastmovementphase_state_pseudoorder_dialogue_chunk")

dict_keys(['S1901M', 'F1901M', 'W1901A', 'S1902M', 'F1902M', 'F1902R', 'W1902A', 'S1903M', 'S1903R', 'F1903M', 'F1903R', 'W1903A', 'S1904M', 'F1904M', 'F1904R', 'W1904A', 'S1905M', 'S1905R', 'F1905M', 'F1905R', 'W1905A', 'S1906M', 'F1906M', 'F1906R', 'W1906A', 'S1907M', 'S1907R', 'F1907M', 'F1907R', 'W1907A', 'S1908M', 'S1908R', 'F1908M', 'F1908R', 'W1908A', 'S1909M', 'S1909R', 'F1909M', 'F1909R', 'W1909A', 'S1910M'])

In [33]:
seqs["S1905M"]["ITALY"][0]["output"]

'S1905M\nItaly -> Russia: Would you be fine with me in Smyrna? [EO_M]'

In [74]:
seqs["S1906M"]["FRANCE"][0]["output"]

'S1906M\nFrance -> Russia: Well played to you man, not over yet but very good strategic play [EO_M]'

In [55]:
seqs.keys()


dict_keys(['S1901M', 'F1901M', 'W1901A', 'S1902M', 'S1902R', 'F1902M', 'F1902R', 'W1902A', 'S1903M', 'S1903R', 'F1903M', 'F1903R', 'W1903A', 'S1904M', 'S1904R', 'F1904M', 'F1904R', 'W1904A', 'S1905M', 'S1905R', 'F1905M', 'F1905R', 'W1905A', 'S1906M', 'S1906R', 'F1906M', 'F1906R', 'W1906A', 'S1907M', 'S1907R', 'F1907M', 'F1907R', 'W1907A', 'S1908M', 'F1908M', 'F1908R', 'W1908A', 'S1909M', 'F1909M'])

In [54]:
seqs["S1901M"]["FRANCE"][0]["output"]


'S1901M\nFrance -> Russia: I like the sound of it! [EO_M]'

In [22]:
game = Game.from_json(open("./data/cicero_redacted_games/game_433762_ITALY_AEFT.json").read())

game_json.keys()

dict_keys(['id', 'is_full_press', 'map', 'phases', 'scoring_system', 'version'])